In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
hf_model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')

# Suppress warning
hf_model.config.pad_token_id = hf_model.config.eos_token_id

device = "cpu"
hf_model.to(device);

In [ ]:
import gpt2
import mlx.core as mx
mlx_model_path = "" # TODO: path to your model
mlx_model = gpt2.load_model(mlx_model_path)

In [ ]:
def tokenize_prompt(prompt):
    encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
    return encoded_input['input_ids']

def decode_output(tokens):
    return tokenizer.decode(tokens)

def generate_hf(prompt, max_new_tokens=1000):
    x = tokenize_prompt(prompt)
    x = x.expand(1, -1)
    y = hf_model.generate(x, max_new_tokens=max_new_tokens, do_sample=False, temperature=1.0)
    return decode_output(y[0])

def generate_mlx(prompt, max_new_tokens=1000):
    x = mx.array(tokenize_prompt(prompt).numpy())

    ys = [x.reshape(-1)]
    count = 0
    for y in mlx_model.generate(x, temperature=1.0):
        ys.append(y)
        count += 1
        if count == max_new_tokens:
            break

    tokens = mx.concatenate(ys, axis=0)
    return decode_output([t.item() for t in tokens])

In [ ]:
prompt = "The city was "

In [ ]:
generate_hf(prompt, max_new_tokens=100)

In [ ]:
generate_mlx(prompt, max_new_tokens=100)